In [1]:
from numba import cuda,float64

import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from PIL import Image
import cv2
import math
import operator
print(cuda.gpus)

<Managed Device 0>


In [ ]:
cell_size = (8, 8)
block_size = (2, 2)
img = Image.open('./Picture/cat-1.jpg')
new_width  =  img.size[0] // cell_size[0] * cell_size[0]
new_height =  img.size[1] // cell_size[1] * cell_size[1]
img = img.resize((new_width,new_height), Image.LANCZOS)
# img = img.resize((64,128), Image.LANCZOS)
img_pixel = np.array(img)
plt.imshow(img_pixel)

In [ ]:
@cuda.jit
def gray_kernel(picture_in, gray_out):    
    row, col = cuda.grid(2)
    image_rows, image_cols,channel = picture_in.shape
    if (row >= image_rows) or (col >= image_cols): 
        return
    rgb                = picture_in[row][col]
    gray_out[row][col] = 0.299*rgb[0] + 0.587*rgb[1] + 0.114*rgb[2]
    # assuming x and y inputs are same length

In [ ]:
# Memory Allocation
out_host   = np.empty([pixel.shape[0], pixel.shape[1]],dtype = pixel.dtype)
x_device   = cuda.to_device(pixel)
height     = cuda.to_device(pixel.shape[0])
width      = cuda.to_device(pixel.shape[1])
out_device = cuda.device_array_like(out_host)

In [ ]:
threadsperblock = (32, 32)
blockspergrid_x = math.ceil(pixel.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(pixel.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)
gray_kernel[blockspergrid, threadsperblock](x_device, out_device)

In [ ]:
out_host = out_device.copy_to_host()

In [ ]:
out_host

In [ ]:
plt.imshow(out_host,cmap='gray')

In [ ]:
x_Dim = threadsperblock[0]//cell_size[0]
y_Dim = threadsperblock[1]//cell_size[1]
print(x_Dim, y_Dim)
sbin_1 = 9
@cuda.jit
def hist_kernel(direction, magnitude, cell_size, d_sbin, result_out):    
    cur_r, cur_c  = cuda.grid(2)
    height, width = direction.shape

    idy   = int(cur_r//cell_size[0])
    idx   = int(cur_c//cell_size[1])
    sbin  = d_sbin
    
    # Creat 3D shared array
    shared     = cuda.shared.array(shape=(x_Dim, y_Dim, sbin_1), dtype=float64)
    idy_shared = idy%x_Dim
    idx_shared = idx%y_Dim
    if cur_r%cell_size[0] == 0 and cur_c%cell_size[1]==0:
        for index in range(sbin_1):
            shared[idy_shared][idx_shared][index] = 0.0

    cuda.syncthreads()
    # kiểm tra
    if cur_r>=height or cur_c>= width:
        return 
    thread_direction = direction[cur_r][cur_c]
    thread_mag       = magnitude[cur_r][cur_c]
    # chia lấy phần nguyên và phần dư
    quotient  = int(thread_direction//sbin)
    remainder =     thread_direction % sbin

    if remainder==0:
        cuda.atomic.add(shared, (idy_shared, idx_shared, quotient), thread_mag)
    else:
        first_bin = quotient

        second_bin     = first_bin+1
        need_to_add    = thread_mag*((second_bin*sbin - thread_direction)/(second_bin*sbin - first_bin*sbin))
        cuda.atomic.add(shared, (idy_shared, idx_shared, first_bin), need_to_add)

            
        second_bin_idx = second_bin
        if second_bin > 8:
            second_bin_idx = 0
        need_to_add_2  = thread_mag*((thread_direction - first_bin*sbin)/(second_bin*sbin - first_bin*sbin))   
        cuda.atomic.add(shared, (idy_shared, idx_shared, second_bin_idx), need_to_add_2)

    
    cuda.syncthreads()

    if cur_r%cell_size[0] == 0 and cur_c%cell_size[1]==0:
        for index in range(sbin_1):
            result_out[idy][idx][index] = shared[idy_shared][idx_shared][index]   
